In [ ]:
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd
import numpy as np
import numba as nb
import imageio
import os
import shutil

randomSeed = 100

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(
    style="ticks", 
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
#plt.rcParams['animation.ffmpeg_path'] = "/opt/conda/bin/ffmpeg"

from main import *
from multiprocessing import Pool
import pandas as pd

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("cmap", colors)
# cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])

colors = ["#403990", "#80A6E2", "#F46F43", "#CF3D3E"]
cmap2 = mcolors.LinearSegmentedColormap.from_list("cmap2", colors)

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.hsv(np.linspace(0, 1, 256)) * 0.85, N=256
)

SAVE_PATH = "./data"
MP4_PATH = "./mp4"
LOCAL_FIG_PATH = "./figs"

In [ ]:
model = PhaseLagPatternFormation(
    strengthK=20, distanceD0=1, phaseLagA0=0.6 * np.pi,
    # initPhaseTheta=np.zeros(1000), 
    omegaMin=0, deltaOmega=0,
    dt=0.001,
    tqdm=True, savePath=SAVE_PATH, shotsnaps=10, 
    randomSeed=9, overWrite=True
)

In [ ]:
model = PhaseLagPatternFormation(
    strengthK=20, distanceD0=1, phaseLagA0=0.6 * np.pi,
    # initPhaseTheta=np.zeros(1000), 
    omegaMin=0, deltaOmega=0,
    agentsNum=120, dt=0.001,
    tqdm=True, savePath=None, shotsnaps=10, 
    randomSeed=9, overWrite=True
)

In [ ]:
model.run(10)

In [ ]:
model.plot(colorsBy="phase")

In [ ]:
sa = StateAnalysis(model)

In [ ]:
sa.plot_spatial()

In [ ]:
classes, centers = sa.calc_classes_and_centers(classDistance=0.5, lookIdx=-1)
tailDotPhaseThetas = np.array([sa.calc_dot_theta(lookIdx=i) for i in np.arange(-1000, 0, 10)])
effectiveFreqs = tailDotPhaseThetas.mean(axis=0)

classCenters = np.array([centers[c].mean(axis=0) for c in classes])
periodicEdges, adjClassCenters = sa.calc_nearby_edges(edgeLenThres=1.5, classCenters=classCenters)

classPosPerent = list()
classNegPerent = list()
classMeanPosFreq = list()
classMeanNegFreq = list()
classMeanFreq = list()
classMeanEffFreq = list()
classMaxRadius = list()

for classIdx, oscIdx in enumerate(classes):
    freqOmega: np.ndarray = sa.model.freqOmega[oscIdx]

    posPerent = (freqOmega >= 0).mean()
    negPerent = (freqOmega < 0).mean()
    
    meanPosFreq = freqOmega[freqOmega >= 0].mean() if np.any(freqOmega >= 0) else 0
    meanNegFreq = freqOmega[freqOmega < 0].mean() if np.any(freqOmega < 0) else 0

    meanFreq = freqOmega.mean()
    meanEffFreq = effectiveFreqs[oscIdx].mean()

    maxRadius = sa.calc_replative_distance(sa.totalPositionX[-1000:, oscIdx], 
                                           classCenters[classIdx]).max()
    
    classPosPerent.append(posPerent)
    classNegPerent.append(negPerent)
    classMeanPosFreq.append(meanPosFreq)
    classMeanNegFreq.append(meanNegFreq)
    classMeanFreq.append(meanFreq)
    classMeanEffFreq.append(meanEffFreq)
    classMaxRadius.append(maxRadius)

classPosPerent = np.array(classPosPerent)
classNegPerent = np.array(classNegPerent)
classMeanPosFreq = np.array(classMeanPosFreq)
classMeanNegFreq = np.array(classMeanNegFreq)

In [ ]:
# triangleCenters = list()
# triangleNodes = list()
# edgeLenThres = 1.5

# for baseNode, classCenter in enumerate(adjClassCenters):
#     relativeDistance = np.linalg.norm(classCenter - adjClassCenters, axis=-1)
#     nearbyClassIdxs = np.where(
#         (relativeDistance < edgeLenThres) & (relativeDistance > 0.1)
#     )[0]
#     delta = sa.model._delta_x(adjClassCenters[nearbyClassIdxs], classCenter, 
#                               sa.model.boundaryLength, 
#                               sa.model.halfBoundaryLength)
#     spaceAngle = np.arctan2(delta[:, 1], delta[:, 0])
#     sortedClassIdxs = nearbyClassIdxs[np.argsort(spaceAngle)]
#     for i in range(len(sortedClassIdxs)):
#         nearNode1 = sortedClassIdxs[i]
#         nearNode2 = sortedClassIdxs[(i + 1) % len(sortedClassIdxs)]
#         nearNode1Pos = adjClassCenters[nearNode1]
#         nearNode2Pos = adjClassCenters[nearNode2]
#         if set([baseNode, nearNode1, nearNode2]) in triangleNodes:
#             continue
#         triangleNodes.append(set([baseNode, nearNode1, nearNode2]))
#         triangleCenters.append((classCenter + nearNode1Pos + nearNode2Pos) / 3)

# triangleCenters = np.array(triangleCenters)
# triangleCenters = triangleCenters[
#     (0 < triangleCenters[:, 0]) & (triangleCenters[:, 0] < sa.model.boundaryLength) &
#     (0 < triangleCenters[:, 1]) & (triangleCenters[:, 1] < sa.model.boundaryLength)
# ]
# triCenEdges, adjTriCenters = sa.calc_nearby_edges(edgeLenThres=1, classCenters=triangleCenters)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

# for edge in triCenEdges[:]:
#     plt.plot(adjTriCenters[edge, 0], adjTriCenters[edge, 1], 
#              color="gray", lw=1, alpha=1, linestyle="-", zorder=0)
# plt.scatter(
#     triangleCenters[:, 0], triangleCenters[:, 1], 
#     facecolor="white", edgecolor="gray", lw=1, s=10, alpha=1, zorder=100
# )
(0, (5, 2, 1, 2))
for edge in periodicEdges[:]:
    plt.plot(adjClassCenters[edge, 0], adjClassCenters[edge, 1], 
             color="#80A6E2", lw=1.5, alpha=0.6, linestyle=(0, (15, 2)), zorder=0)
plt.scatter(
    classCenters[:, 0], classCenters[:, 1], 
    facecolor="white", cmap=cmap, s=30, edgecolor="#80A6E2", lw=1.5
)

sa.plot_spatial(ax=ax, colorsBy="phase")

xShift = 0.5
plt.xlim(0 + xShift, sa.model.boundaryLength + xShift)
plt.ylim(0, sa.model.boundaryLength)
plt.xticks(
    np.arange(0 + xShift, sa.model.boundaryLength + xShift + 1),
    np.arange(0, sa.model.boundaryLength + 1))
plt.tick_params(length=3, direction="in")

plt.savefig(f"{LOCAL_FIG_PATH}/achiral_snapshot.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

plt.scatter(
    classCenters[:, 0] - 0.003, classCenters[:, 1] + 0.008, 
    facecolor="white", cmap=cmap, s=650, edgecolor="black", lw=4
)
plt.scatter(
    classCenters[:, 0], classCenters[:, 1], 
    facecolor="white", cmap=cmap, s=30, edgecolor="black", lw=1.5, zorder=10
)

for edge in periodicEdges[:]:
    plt.plot(adjClassCenters[edge, 0], adjClassCenters[edge, 1], 
             color="black", lw=1.2, alpha=0.5, linestyle="--", zorder=0)

width = 0.4
height = 0.4
linewidth = 10
textLength = 0.17
posTextAngle = np.pi * 0.5
negTextAngle = np.pi * 1.5

for classIdx in range(len(classes)):
    posPerent = classPosPerent[classIdx]
    negPerent = classNegPerent[classIdx]
    meanPosFreq = classMeanPosFreq[classIdx]
    meanNegFreq = classMeanNegFreq[classIdx]
    classCenter = classCenters[classIdx]
    angle = 90 - 360 * posPerent / 2
    ax.add_patch(patches.Arc(xy=classCenter, width=width, height=height,  
                             theta1=angle, theta2=angle + 360 * posPerent, 
                            #  color=cmap((meanPosFreq - min(classMeanPosFreq)) / (max(classMeanPosFreq) - min(classMeanPosFreq)) / 2 + 0.5), 
                             color="red",
                             alpha=1, lw=linewidth))
    ax.add_patch(patches.Arc(xy=classCenter, width=width, height=height, 
                             theta1=angle + 360 * posPerent, theta2=angle + 360, 
                            #  color=cmap((meanNegFreq - min(classMeanNegFreq)) / (max(classMeanNegFreq) - min(classMeanNegFreq)) / 2), 
                             color="#403990",
                             alpha=1, lw=linewidth))
    ax.text(classCenter[0] + textLength * np.cos(posTextAngle),
            classCenter[1] + textLength * np.sin(posTextAngle),
            f"{meanPosFreq:.2f}", ha="center", va="center", fontsize=6, color="black")
    ax.text(classCenter[0] + textLength * np.cos(negTextAngle),
            classCenter[1] + textLength * np.sin(negTextAngle),
            f"{meanNegFreq:.2f}", ha="center", va="center", fontsize=6, color="white")

xShift = 0.5
plt.xlim(0 + xShift, sa.model.boundaryLength + xShift)
plt.ylim(0, sa.model.boundaryLength)
plt.xticks(
    np.arange(0 + xShift, sa.model.boundaryLength + xShift + 1),
    np.arange(0, sa.model.boundaryLength + 1))
plt.tick_params(length=3, direction="in")

plt.savefig(f"{LOCAL_FIG_PATH}/statistics.pdf", bbox_inches="tight")

In [ ]:
edgeLenThres = 1.5

nearbyMeanFreqs = dict()

for i in range(len(classes)):
    selectClassFreq = classMeanFreq[i]
    nearbyMeanFreqs[selectClassFreq] = list()
    relativeDistance = sa.calc_replative_distance(classCenters[i], classCenters)
    nearbyClassIdx = np.where(
        (relativeDistance < edgeLenThres) & (relativeDistance > 0.1)
    )[0]
    for j in nearbyClassIdx:
        nearbyMeanFreqs[selectClassFreq].append(classMeanFreq[j])

xFreqs = list()
yFreqs = list()

for k, v in nearbyMeanFreqs.items():
    xFreqs += [k] * len(v)
    yFreqs += v

In [ ]:
meanNearByFreq = [np.mean(v) for v in nearbyMeanFreqs.values()]
plt.scatter(nearbyMeanFreqs.keys(), meanNearByFreq)

In [ ]:
sc = plt.scatter(xFreqs, yFreqs, s=10, c=xFreqs, cmap=cmap)
plt.colorbar(sc, label="Mean Frequency", fraction=0.05, pad=0.02)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

bound = np.abs(classMeanFreq).max()

sc = plt.scatter(
    classCenters[:, 0], classCenters[:, 1], 
    c=classMeanFreq, s=200, edgecolor="black", lw=1.5, zorder=10,
    cmap=cmap, vmin=-bound, vmax=bound
)
plt.colorbar(sc, label="Mean Frequency", orientation="vertical", pad=0.02)

for edge in periodicEdges[:]:
    plt.plot(adjClassCenters[edge, 0], adjClassCenters[edge, 1], 
             color="black", lw=1.2, alpha=0.5, linestyle="--", zorder=0)
    
for i, classCenter in enumerate(classCenters):
    plt.text(
        classCenter[0] + 0.01, classCenter[1] + 0.1, 
        f"{i}", ha="left", va="bottom", fontsize=12, color="black"
    )

xShift = 0.5
plt.xlim(0 + xShift, sa.model.boundaryLength + xShift)
plt.ylim(0, sa.model.boundaryLength)
plt.xticks(
    np.arange(0 + xShift, sa.model.boundaryLength + xShift + 1),
    np.arange(0, sa.model.boundaryLength + 1))
plt.tick_params(length=3, direction="in")
plt.savefig(f"{LOCAL_FIG_PATH}/mean_freq.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

bounds = np.abs(classMeanEffFreq).max()

sc = plt.scatter(
    classCenters[:, 0], classCenters[:, 1], 
    c=classMeanEffFreq, s=200, edgecolor="black", lw=1.5, zorder=10,
    cmap=cmap, vmin=-bounds, vmax=bounds
)
# sc = plt.scatter(-np.ones(len(classes)*2), -np.ones(len(classes)*2), cmap=cmap,
#                  c=np.concatenate([classMeanEffFreq, -np.array(classMeanEffFreq)]))
plt.colorbar(sc, label="Mean Effective Frequency", orientation="vertical", pad=0.02)

for edge in periodicEdges[:]:
    plt.plot(adjClassCenters[edge, 0], adjClassCenters[edge, 1], 
             color="black", lw=1.2, alpha=0.5, linestyle="--", zorder=0)
    
for i, classCenter in enumerate(classCenters):
    plt.text(
        classCenter[0] + 0.01, classCenter[1] + 0.1, 
        f"{i}", ha="left", va="bottom", fontsize=12, color="black"
    )

xShift = 0.5
plt.xlim(0 + xShift, sa.model.boundaryLength + xShift)
plt.ylim(0, sa.model.boundaryLength)
plt.xticks(
    np.arange(0 + xShift, sa.model.boundaryLength + xShift + 1),
    np.arange(0, sa.model.boundaryLength + 1))
plt.tick_params(length=3, direction="in")
plt.savefig(f"{LOCAL_FIG_PATH}/mean_eff_freq.pdf", bbox_inches="tight")

In [ ]:
edgeLenThres = 1.5

highPosClassIdx = np.arange(len(classes))[classPosPerent > 0.6]
nearbyPosPerent = list()
nearbyNegPerent = list()

for classIdx in highPosClassIdx:
    relativeDistance = sa.calc_replative_distance(
        position1=classCenters[classIdx], position2=classCenters
    )
    nearbyClassIdx = np.where(
        (relativeDistance < edgeLenThres) & (relativeDistance > 0.1)
    )[0]
    nearbyNegPerent += classNegPerent[nearbyClassIdx].tolist()
    nearbyPosPerent += classPosPerent[nearbyClassIdx].tolist()

print("Nearby positive parent percentage:", np.mean(nearbyPosPerent))
print("Nearby negative parent percentage:", np.mean(nearbyNegPerent))

highNegClassIdx = np.arange(len(classes))[classNegPerent > 0.6]
nearbyPosPerent = list()
nearbyNegPerent = list()

for classIdx in highNegClassIdx:
    relativeDistance = sa.calc_replative_distance(
        position1=classCenters[classIdx], position2=classCenters
    )
    nearbyClassIdx = np.where(
        (relativeDistance < edgeLenThres) & (relativeDistance > 0.1)
    )[0]
    nearbyNegPerent += classNegPerent[nearbyClassIdx].tolist()
    nearbyPosPerent += classPosPerent[nearbyClassIdx].tolist()

print("Nearby positive parent percentage:", np.mean(nearbyPosPerent))
print("Nearby negative parent percentage:", np.mean(nearbyNegPerent))

In [ ]:
edges = sa.calc_nearby_edges(edgeLenThres=1.5, 
                             classCenters=classCenters,
                             relativeDistance=True)

distances = pd.DataFrame(columns=["centerDis", "radius1", "radius2"])

for edge in edges:
    distances = distances.append({
        "centerDis": sa.calc_replative_distance(
            classCenters[edge[0]], classCenters[edge[1]]
        ),
        "radius1": classMaxRadius[edge[0]],
        "radius2": classMaxRadius[edge[1]]
    }, ignore_index=True)

distances = (
    distances.sort_values(by="centerDis")
    .reset_index(drop=True)
)

In [ ]:
for idx, row in distances.iterrows():
    plt.plot(
        [0, row["centerDis"]], [idx, idx],
        markersize=5, color="C0", lw=1.5
    )
    plt.plot(
        [row["centerDis"] - max(row["radius1"], row["radius2"]), row["centerDis"]], [idx, idx],
        markersize=5, color="C1", lw=1.5
    )

plt.axvline(x=1, color="black", lw=1.5, linestyle="--")

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xlim(0, 1.5)
plt.ylim(0, len(distances))
plt.tick_params(length=3, direction="in")

In [ ]:
classCoordinate = pd.DataFrame(
    classCenters,
    columns=["x", "y"]
).round()
drawnClasses = classCoordinate.sort_values(by=["y", "x"]).index

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

for edge in periodicEdges[:]:
    plt.plot(adjClassCenters[edge, 0], adjClassCenters[edge, 1], 
             color="black", lw=1.2, alpha=0.3, linestyle="--", zorder=0)

plt.scatter(
    classCenters[:, 0], classCenters[:, 1], 
    facecolor="white", cmap=cmap, s=30, edgecolor="black", lw=1.5
)

for i, drawnClass in enumerate(drawnClasses):
    plt.text(
        classCenters[drawnClass][0] + 0.01, 
        classCenters[drawnClass][1] + 0.1, 
        f"{i}", ha="left", va="bottom", fontsize=12, color="black"
    )

sa.plot_spatial(ax=ax, colorsBy="phase")

xShift = 0.5
plt.xlim(0 + xShift, sa.model.boundaryLength + xShift)
plt.ylim(0, sa.model.boundaryLength)
plt.xticks(
    np.arange(0 + xShift, sa.model.boundaryLength + xShift + 1),
    np.arange(0, sa.model.boundaryLength + 1))
plt.tick_params(length=3, direction="in")


In [ ]:
sortedOscIdxs = np.concatenate([classes[c] for c in drawnClasses])
sortedPos = sa.totalPositionX[-1, sortedOscIdxs]
relativeDistance = sa.calc_replative_distance(
    sortedPos, sortedPos[:, np.newaxis]
)
A = (relativeDistance <= sa.model.distanceD0).astype(np.int32)

In [ ]:
plt.imshow(A, cmap=cmap)

In [ ]:
import matplotlib.patches as patches
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import product
import pandas as pd
import numpy as np
import numba as nb
import imageio
import os
import shutil

randomSeed = 100

%matplotlib inline
%config InlineBackend.figure_format = "retina"

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(
    style="ticks", 
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
#plt.rcParams['animation.ffmpeg_path'] = "/opt/conda/bin/ffmpeg"

from main import *
from multiprocessing import Pool
import pandas as pd

colors = ["#403990", "#80A6E2", "#FBDD85", "#F46F43", "#CF3D3E"]
cmap = mcolors.LinearSegmentedColormap.from_list("cmap", colors)
# cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])

colors = ["#403990", "#80A6E2", "#F46F43", "#CF3D3E"]
cmap2 = mcolors.LinearSegmentedColormap.from_list("cmap2", colors)

SAVE_PATH = r"E:\MS_ExperimentData\general"
MP4_PATH = r"E:\MS_ExperimentData\mp4"

# SAVE_PATH = r"D:\MS_ExperimentData\general"
# MP4_PATH = r"D:\MS_ExperimentData\mp4"

# LOCAL_FIG_PATH = "./PCT_C_figs"
LOCAL_FIG_PATH = "./figs"

In [ ]:
model = PhaseLagPatternFormation1D(strengthK=20, distanceD0=1, phaseLagA0=0.6*np.pi, 
                                  dt=0.001,
                                  tqdm=True, savePath=SAVE_PATH, shotsnaps=10, 
                                  randomSeed=9, overWrite=True)
model.run(80000)